In [1]:
# User-definable parameters
parameters = {
    "fasta_file_name": "data/190912_BovineHistone_cRAP_MPDS_01.fasta",
    "max_pep_len": 20,
    "charges": [2, 3],
    "ppm": 10,
    "isotopes": [0, 1],
    "explicit_isoform_format": "prop-{}-OH",
    "variable_mods": {
        'me': ['K', "R"],
        'me2': ['K'],
        'me3': ['K'],
        'ac': ['K'],
        'fo': ['K'],
    },
    "MGF_file_names": {
        "light_c13": "data/mgf/LightC_KSAPAT.mgf",
        "light_deuterium": "data/mgf/LightD_KSAPAT.mgf",
        "c13": "data/mgf/HeavyC_KSAPAT.mgf",
        "deuterium": "data/mgf/HeavyD_KSAPAT.mgf",   
    },
    "CSV_file_names": {
        "c13": "data/CSV/HeavyC13_2.csv",
        "deuterium": "data/CSV/HeavyD_2.csv",
    }
}

In [2]:
import pyteomics
import pyteomics.fasta
import pyteomics.parser
import pyteomics.electrochem
import pyteomics.mass
import pyteomics.mgf
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# import sympy
# import csv
# from collections import Counter
# from time import asctime



def setAAComp(fixed_ptm_type):
    aa_comp = {
        key: pyteomics.mass.Composition(value) for key, value in pyteomics.mass.std_aa_comp.items()
    }
    if fixed_ptm_type == "light":
        fixed_ptm = pyteomics.mass.Composition('C3H4O1')
    elif fixed_ptm_type == "c13":
        fixed_ptm = pyteomics.mass.Composition('C[13]3H4O1')
    elif fixed_ptm_type == "deuterium":
        fixed_ptm = pyteomics.mass.Composition('C3H[2]5H-1O1')
    elif fixed_ptm_type == "none":
        fixed_ptm = pyteomics.mass.Composition('')
#     elif fixed_ptm_type == "heavy_acetyl":
#         fixed_ptm = mass.Composition('C2H[2]3O1H-1')
    else:
        raise Exception("Undefined fixed_ptm_type")
    aa_comp['K'] += fixed_ptm
    aa_comp['prop-'] = fixed_ptm + pyteomics.mass.Composition("H1")
    aa_comp.update(
        {
            'me': pyteomics.mass.Composition('C1H2'),
            'me2': pyteomics.mass.Composition('C2H4') - fixed_ptm,
            'me3': pyteomics.mass.Composition('C3H6') - fixed_ptm,
            'fo': pyteomics.mass.Composition('C1O1') - fixed_ptm,
            'ac': pyteomics.mass.Composition('C2H2O1') - fixed_ptm,
        }
    )
    return aa_comp


def getIsoforms(parameters, description_must_contain="istone", protease="arg-c"):
    fasta_file = pyteomics.fasta.FASTA(parameters["fasta_file_name"])
    peptides = set()
    isoforms = set()
    for description, sequence in fasta_file:
        if description_must_contain not in description:
            continue
        new_peptides = pyteomics.parser.cleave(
            sequence,
            pyteomics.parser.expasy_rules[protease]
        )
        peptides.update(
            {
                pep for pep in new_peptides if len(pep) < parameters["max_pep_len"]
            }
        )
    isoforms = getIsoformsOfPeptides(
        peptides,
        parameters["explicit_isoform_format"],
        parameters["variable_mods"]
    )
    return isoforms


def getIsoformsOfPeptides(peptides, isoform_format, var_mods):
    isoforms = {
        isoform_format.format(
            isoform
        ) for peptide in peptides for isoform in pyteomics.parser.isoforms(
            peptide,
            variable_mods=var_mods
        )
    }
    return isoform


def getIsoformMasses(parameters, isoforms, fixed_ptm_type, isotope_mass=1.0028):
    masses = np.sort(
        [
            pyteomics.mass.calculate_mass(
                sequence=isoform,
                charge=charge,
                aa_comp=setAAComp(fixed_ptm_type)
            ) + isotope_mass * isotope / charge for isoform in isoforms for charge in parameters['charges'] for isotope in parameters['isotopes']
        ]
    )
    return masses


def getMZRanges(mass_error, masses, relative=True, mass_frequencies=None):
    if relative:
        lower_masses = masses / (1 + mass_error / 10**6)
        upper_masses = masses * (1 + mass_error / 10**6)
    else:
        lower_masses = masses - mass_error
        upper_masses = masses + mass_error
    all_masses = np.concatenate([lower_masses, upper_masses])
    if mass_frequencies is None:
        frequencies = np.ones(len(all_masses), dtype=np.int)
        frequencies[len(frequencies) // 2:] = -1
    else:
        frequencies = np.zeros(len(all_masses), dtype=np.int)
        frequencies[:len(frequencies) // 2] = mass_frequencies
        frequencies[len(frequencies) // 2:] = - mass_frequencies
    order = np.argsort(all_masses)
    all_masses = all_masses[order]
    frequencies = np.cumsum(frequencies[order])
    mz_range = np.diff(all_masses)
    selection = mz_range > 0
    select_masses = all_masses[:-1][selection]
    select_mz_range = mz_range[selection]
    select_frequencies = frequencies[:-1][selection]
    selection = select_frequencies > 0
    mz_ranges = np.stack(
        [
            select_masses,
            select_mz_range,
            select_frequencies,
        ]
    ).T[selection]
    return mz_ranges


def plotMzRanges(mz_ranges, mirrored=False, normalized=False):
    left_masses = mz_ranges[:, 0]
    right_masses = mz_ranges[:, 0] + mz_ranges[:, 1]
    right_masses = right_masses[~np.isin(right_masses, left_masses)]
    masses = np.concatenate([left_masses, right_masses])
    frequencies = np.zeros(len(masses), dtype=np.float)
    frequencies[:mz_ranges.shape[0]] = mz_ranges[:, 2]
    order = np.argsort(masses)
    masses = masses[order]
    frequencies = frequencies[order]
    if normalized:
        frequencies /= np.max(frequencies)
    if mirrored:
        frequencies *= -1
    plt.step(masses, frequencies, where="post")


def getAmbiguityIndex(mz_ranges):
    ranges = mz_ranges[:, 1]
    frequencies = mz_ranges[:, 2]
#     ambiguity_index = np.sum(frequencies**2 * ranges / np.sum(ranges * frequencies))
    ambiguity_index = np.sum(frequencies * ranges / np.sum(ranges))
    return ambiguity_index

In [ ]:
isoforms = getIsoforms(parameters)
mass_error = parameters['ppm']

light_masses = getIsoformMasses(parameters, isoforms, "light")
light_mz_ranges = getMZRanges(mass_error, light_masses)

c13_masses = getIsoformMasses(parameters, isoforms, "c13")
c13_mz_ranges = getMZRanges(mass_error, c13_masses)

deuterium_masses = getIsoformMasses(parameters, isoforms, "deuterium")
deuterium_mz_ranges = getMZRanges(mass_error, deuterium_masses)

In [ ]:
%matplotlib notebook
plotMzRanges(light_mz_ranges)
plotMzRanges(c13_mz_ranges)
plotMzRanges(deuterium_mz_ranges)
plt.legend(["light", "c13", "deuterium"])
print("Isoforms: {}\n".format(len(isoforms)))
print("-" * 50)
print("PTM\t\tMZ range\tAmbiguity index")
print("-" * 50)
for fixed_ptm, mz_ranges in [
    ("light\t", light_mz_ranges),
    ("c13\t", c13_mz_ranges),
    ("deuterium", deuterium_mz_ranges),
]:
    print(
        "{}\t{:.3f}\t\t{:.3f}".format(
            fixed_ptm,
            np.sum(mz_ranges[:, 1]),
            getAmbiguityIndex(mz_ranges),
        )
    )
print("-" * 50)

In [ ]:
def getMZsFromMGF(mgf_file_name):
    precursor_masses = np.array(
        [
            spectrum['params']['pepmass'][0] for spectrum in pyteomics.mgf.MGF(mgf_file_name)
        ]
    )
    masses = np.stack(
        [
            *np.unique(precursor_masses, return_counts=True)
        ]
    ).T
    return masses


def getHitCounts(masses, mz_ranges):
    mgf_masses = masses[:, 0]
    mgf_frequencies = masses[:, 1]
    indices = np.searchsorted(
        mz_ranges[:, 0],
        mgf_masses,
    ) - 1
    hits = mgf_masses < (
        mz_ranges[indices, 0] + mz_ranges[indices, 1]
    )
    hit_counts = np.zeros(len(mgf_masses), dtype=int)
    hit_counts[hits] = mz_ranges[indices[hits], 2]
    return hit_counts


def getAmbiguity(hit_counts, frequencies):
    ambiguity = np.sum(hit_counts * frequencies) / np.sum((hit_counts > 0) * frequencies)
    return ambiguity

In [ ]:
light_c13_mgf_masses = getMZsFromMGF(parameters["MGF_file_names"]["light_c13"])
light_deuterium_mgf_masses = getMZsFromMGF(parameters["MGF_file_names"]["light_deuterium"])
c13_mgf_masses = getMZsFromMGF(parameters["MGF_file_names"]["c13"])
deuterium_mgf_masses = getMZsFromMGF(parameters["MGF_file_names"]["deuterium"])

light_c13_hit_counts = getHitCounts(light_c13_mgf_masses, light_mz_ranges)
light_deuterium_hit_counts = getHitCounts(light_deuterium_mgf_masses, light_mz_ranges)
c13_hit_counts = getHitCounts(c13_mgf_masses, c13_mz_ranges)
deuterium_hit_counts = getHitCounts(deuterium_mgf_masses, deuterium_mz_ranges)

In [ ]:
print("-" * 65)
print("{:<20} {:<15} {:<15} {}".format("PTM", "Precursors", "Annotatable" ,"Ambiguity"))
print("-" * 65)
for fixed_ptm, hit_counts, frequencies in [
    ("light_c13", light_c13_hit_counts, light_c13_mgf_masses[:, 1]),
    ("light_deuterium", light_deuterium_hit_counts, light_deuterium_mgf_masses[:, 1]),
    ("c13", c13_hit_counts, c13_mgf_masses[:, 1]),
    ("deuterium", deuterium_hit_counts, deuterium_mgf_masses[:, 1]),
]:
    print(
        "{:<20} {:<15} {:<15} {:.3f}".format(
            fixed_ptm,
            np.sum(frequencies.astype(np.int)),
            np.sum((frequencies * (hit_counts > 0)).astype(np.int)),
            getAmbiguity(hit_counts, frequencies)
        )
    )
print("-" * 65)

In [ ]:
# %matplotlib notebook
# light_c13_experimental_mz_ranges = getMZRanges(
#     mass_error=0.7,
#     masses=light_c13_mgf_masses[:,0],
#     relative=False,
#     mass_frequencies=light_c13_mgf_masses[:,1]
# )
# plotMzRanges(light_mz_ranges, normalized=True)
# plotMzRanges(light_c13_experimental_mz_ranges, mirrored=True, normalized=True)

In [ ]:
# %matplotlib notebook
# light_deuterium_experimental_mz_ranges = getMZRanges(
#     mass_error=0.7,
#     masses=light_deuterium_mgf_masses[:,0],
#     relative=False,
#     mass_frequencies=light_deuterium_mgf_masses[:,1]
# )
# plotMzRanges(light_mz_ranges, normalized=True)
# plotMzRanges(light_deuterium_experimental_mz_ranges, mirrored=True, normalized=True)

In [ ]:
# %matplotlib notebook
# c13_experimental_mz_ranges = getMZRanges(
#     mass_error=0.7,
#     masses=c13_mgf_masses[:,0],
#     relative=False,
#     mass_frequencies=c13_mgf_masses[:,1]
# )
# plotMzRanges(c13_mz_ranges, normalized=True)
# plotMzRanges(c13_experimental_mz_ranges, mirrored=True, normalized=True)

In [ ]:
# %matplotlib notebook
# deuterium_experimental_mz_ranges = getMZRanges(
#     mass_error=0.7,
#     masses=deuterium_mgf_masses[:,0],
#     relative=False,
#     mass_frequencies=deuterium_mgf_masses[:,1]
# )
# plotMzRanges(deuterium_mz_ranges, normalized=True)
# plotMzRanges(deuterium_experimental_mz_ranges, mirrored=True, normalized=True)

In [3]:
import sympy
import csv
from collections import Counter
from time import asctime
from sympy.utilities.lambdify import lambdify, implemented_function

x, n = sympy.symbols("x n")

class Feature(object):

    def __init__(
        self,
        sequence,
        ptms_string,
        isotopes,
        isotopic_ratios,
        aa_comp,
        mass_data,
):
        self.sequence = sequence
        self.ptms_string = ptms_string
        self.isotopic_ratios = np.array(isotopic_ratios).astype(float)
        self.negative_isotopes = isotopes
        self.__parsePTMs(ptms_string)
        self.composition = pyteomics.mass.Composition(
            sequence=self.modx,
            aa_comp=aa_comp,
            mass_data=mass_data
        )

    def __parsePTMs(self, ptms_string):
        prev_index = 0
        sequence = ""
        n_term = "H"
        c_term = "OH"
        for raw_ptm in ptms_string.split("|"):
            loc, ptm = raw_ptm.split()
            try:
                loc = int(loc[1:-1])
                if loc == prev_index:
                    if ptm in ["prc", "prd"]:
                        continue
                    else:
                        sequence = sequence[:-3]
                partial_seq = self.sequence[prev_index: loc]
                sequence += partial_seq + ptm
                prev_index = loc
            except:
                if loc == "[N-term]":
                    n_term = ptm
                elif loc == "[C-term]":
                    c_term = ptm
        sequence += self.sequence[prev_index:]
        self.modx = "-".join([n_term, sequence, c_term])

        
    def setFormula(self, isotopic_envelope):
        self.formula = sum(
            isotopic_envelope[i] for i in range(-self.negative_isotopes, 0)
        )
    
    def __str__(self):
        return self.modx
    

def getAACompsAndMassData():
    print("{}: Updating atom masses and PTM compositions".format(asctime()))
    aa_comp = {
        key: pyteomics.mass.Composition(value) for key, value in pyteomics.mass.std_aa_comp.items()
    }
    mass_data = {
        key: {
            key2: (value2[0], value2[1]) for key2, value2 in value.items()
        } for key, value in pyteomics.mass.nist_mass.items()
    }
    mass_data["Heavyh"] = {
        0: (pyteomics.mass.nist_mass["H"][2][0], 1.0),
        1: (pyteomics.mass.nist_mass["H"][1][0], 1 - x),
        2: (pyteomics.mass.nist_mass["H"][2][0], x)
    }
    mass_data["Heavyc"] = {
        0: (pyteomics.mass.nist_mass["C"][13][0], 1.0),
        12: (pyteomics.mass.nist_mass["C"][12][0], 1 - x),
        13: (pyteomics.mass.nist_mass["C"][13][0], x)
    }
    aa_comp["prc"] = pyteomics.mass.Composition(
        {
            "Heavyc": 3,
            "O": 1,
            "H": 4
        }
    )
    aa_comp["prd"] = pyteomics.mass.Composition(
        {
            "C": 3,
            "O": 1,
            "Heavyh": 5,
            "H": -1,
        }
    )
    aa_comp['prc-'] = aa_comp["prc"] + pyteomics.mass.Composition("H1")
    aa_comp['prd-'] = aa_comp["prd"] + pyteomics.mass.Composition("H1")
    aa_comp.update(
        {
            'me': pyteomics.mass.Composition('C1H2'),
            'me2': pyteomics.mass.Composition('C2H4'),
            'me3': pyteomics.mass.Composition('C3H6'),
            'fo': pyteomics.mass.Composition('C1O1'),
            'ac': pyteomics.mass.Composition('C2H2O1'),
        }
    )
    aa_comp['prc+me'] = aa_comp["prc"] + aa_comp["me"]
    aa_comp['prd+me'] = aa_comp["prd"] + aa_comp["me"]
    return aa_comp, mass_data


def getFeatures(in_file_name, aa_comp, mass_data, delimiter=","):
    print("{}: Reading features".format(asctime()))
    features = []
    with open(in_file_name, "r") as raw_infile:
        infile = csv.reader(raw_infile, delimiter=delimiter)
        next(infile)  # header
        for sequence, ptms_string, isotopes, *isotopic_ratios in infile:
            if sequence == "":
                continue
            features.append(
                Feature(
                    sequence,
                    ptms_string,
                    int(isotopes),
                    isotopic_ratios,
                    aa_comp,
                    mass_data,
                )
            )
    return features


def setIsotopicFormulae(features, mass_data):
    envelopes = {}
    min_comp = pyteomics.mass.Composition(features[0].composition)
    for f in features[1:]:
        min_comp &= f.composition
    min_expr = 1 
    for atom, count in min_comp.items():
        monoisotopic_abundance = mass_data[atom][
            round(mass_data[atom][0][0])
        ][1]
        if atom.startswith("Heavy"):
            modifier = -1
        elif atom == "O":
            modifier = 2
        else:
            modifier = 1
        min_expr *= (
            monoisotopic_abundance + (n**modifier) * (
                1 - monoisotopic_abundance
            )
        )**count
    min_expr = sympy.expand(min_expr)
    for feature in features:
        print("{}: Processing feature {}".format(asctime(), feature))
        composition = feature.composition
        composition_tuple = tuple(sorted(composition.items()))
        if composition_tuple in envelopes:
            isotopes = envelopes[composition_tuple]
        else:
            expr = 1
            for atom, count in (composition - min_comp).items():
                monoisotopic_abundance = mass_data[atom][
                    round(mass_data[atom][0][0])
                ][1]
                if atom.startswith("Heavy"):
                    modifier = -1
                elif atom == "O":
                    modifier = 2
                else:
                    modifier = 1
                expr *= (
                    monoisotopic_abundance + (n**modifier) * (
                        1 - monoisotopic_abundance
                    )
                )**count
            expr = sympy.expand(expr)
            expr *= min_expr
            expr = sympy.expand(expr)
            isotopes = {
                i: expr.coeff(n, i) for i in range(
                    -sum(c for i, c in composition.items() if i.startswith("Heavy")),
                    sympy.degree(expr.subs([(x, 1)])) + 1
                )
            }
            envelopes[composition_tuple] = isotopes
        feature.setFormula(isotopes)

In [6]:
# in_file_name = parameters["CSV_file_names"]["c13"]
# in_file_name = parameters["CSV_file_names"]["deuterium"]
aa_comp, mass_data = getAACompsAndMassData()
c13_features = getFeatures(parameters["CSV_file_names"]["c13"], aa_comp, mass_data)
deuterium_features = getFeatures(parameters["CSV_file_names"]["deuterium"], aa_comp, mass_data)

Mon Nov 25 15:34:38 2019: Updating atom masses and PTM compositions
Mon Nov 25 15:34:38 2019: Reading features
Mon Nov 25 15:34:38 2019: Reading features


In [7]:
setIsotopicFormulae(c13_features, mass_data)
setIsotopicFormulae(deuterium_features, mass_data)

Mon Nov 25 15:34:39 2019: Processing feature prc-GKprcGGKprcGLGKprcGGAKprcR-OH
Mon Nov 25 15:34:43 2019: Processing feature prc-GKfoGGKprcGLGKprcGGAKprcR-OH
Mon Nov 25 15:34:59 2019: Processing feature prc-GKfoGGKprcGLGKacGGAKacR-OH
Mon Nov 25 15:35:03 2019: Processing feature prc-GKacGGKprcGLGKacGGAKacR-OH
Mon Nov 25 15:35:07 2019: Processing feature prc-GKprcGGKacGLGKprcGGAKacR-OH
Mon Nov 25 15:35:18 2019: Processing feature prc-GKprcGGKprcGLGKacGGAKprcR-OH
Mon Nov 25 15:35:37 2019: Processing feature prc-GKprcGGKprcGLGKprcGGAKacR-OH
Mon Nov 25 15:35:37 2019: Processing feature prc-GKfoGGKprcGLGKprcGGAKacR-OH
Mon Nov 25 15:35:46 2019: Processing feature prc-Kme2SAPATGGVKprcKprcPHR-OH
Mon Nov 25 15:36:07 2019: Processing feature prc-Kprc+meSAPATGGVKprcKprcPHR-OH
Mon Nov 25 15:36:46 2019: Processing feature prc-KprcSAPATGGVKprcKprcPHR-OH
Mon Nov 25 15:37:22 2019: Processing feature prc-Kme2SAPATGGVKprc+meKprcPHR-OH
Mon Nov 25 15:37:44 2019: Processing feature prc-Kme3SAPATGGVKprcKprcPH

In [35]:
numspace = np.linspace(0.97, 1, 301)

for features in [c13_features, deuterium_features]:
    for feature in features:
        irs = feature.isotopic_ratios
        f = lambdify(x, 1 - feature.formula, "numpy")
        targets = f(numspace)
        distances = targets.reshape(-1,1) - irs.reshape(1,-1)
        errors = np.sum(distances**2, axis=1)
        min_error_index = np.argmin(errors)
        min_error = numspace[min_error_index]
#         print(feature, min_error, f(min_error))#, irs)
        print("{:<40}{:.2f}-{:.3f} {}".format(str(feature), 100*min_error, f(min_error), irs))

prc-GKprcGGKprcGLGKprcGGAKprcR-OH       99.46-0.961 [0.96172427 0.96101725 0.96031419 0.96066082 0.96062104]
prc-GKfoGGKprcGLGKprcGGAKprcR-OH        99.41-0.967 [0.96782738 0.96588898 0.96868529 0.96634188 0.96677959]
prc-GKfoGGKprcGLGKacGGAKacR-OH          99.46-0.985 [0.98220446 0.99016153 0.97825986 0.98629278 0.98894424]
prc-GKacGGKprcGLGKacGGAKacR-OH          99.06-0.975 [0.97629345 0.97541239 0.97435398 0.97363758 0.97430204]
prc-GKprcGGKacGLGKprcGGAKacR-OH         99.51-0.979 [0.97942834 0.97929318 0.98006695 0.97899809 0.97960232]
prc-GKprcGGKprcGLGKacGGAKprcR-OH        99.47-0.971 [0.97142001 0.9695496  0.97016593 0.97132644 0.96950668]
prc-GKprcGGKprcGLGKprcGGAKacR-OH        99.20-0.954 [0.95542437 0.95410351 0.95465631 0.95421414 0.95358981]
prc-GKfoGGKprcGLGKprcGGAKacR-OH         99.48-0.978 [0.97923115 0.97668117 0.97937445 0.97853031 0.97756024]
prc-Kme2SAPATGGVKprcKprcPHR-OH          99.24-0.970 [0.97524065 0.9676954  0.96833099 0.97174323 0.96830247]
prc-Kprc+meSAPATGGV